# Embedded C++ Lec(1)

In [5]:
# include <iostream>

using namespace std;

#### Converting Code from C to C++ ####
Struct Time (Hrs,Mins,Secs)

In [6]:
class Time {
  private :
    int Hours;
    int Mins;
    int Secs;
    
    public :
    Time () = default;
    
    Time (int x, int y, int z)
    {
        this->Hours = x;
        this -> Mins = y;
        this -> Secs = z;
    }
    
    void PrintTime ()
    {
        cout<<"Time is :" << Hours << ":" << Mins << ":" << Secs;
    }
};

In [8]:
int main () {
    Time time(10,01,30);
    time.PrintTime();
}

## Another Method with Sets & Getters

In [15]:
#include <iostream>
using namespace std;

In [16]:
class Time {
    private :
    int Hours;
    int Mins;
    int Secs;
    
    public :
    /* ================================== Setters ===================================== */
    void SetHour (int x)
    {
        this -> Hours = x;
    }
    
    void SetMins (int y)
    {
        this -> Mins = y;
    }
    
    void SetSecs (int z)
    {
        this -> Secs = z;
    }
    /* ================================== Getters ======================================*/
    int GetHour (void)
    {
        return Hours;
    }
    
    int GetMins (void)
    {
        return Mins;
    }
    
    int GetSecs (void)
    {
        return Secs;
    }
    
};

In [18]:
int main ()
{
    Time time;
    
    time.SetHour(10);
    time.SetMins(21);
    time.SetSecs(55);
    
    cout<< time.GetHour() << ":" << time.GetMins() << ":" << time.GetSecs();
    
    
    return 0;
}

### With Operator Overloading

In [ ]:
#include <iostream>
#include <iomanip>

using namespace std;

In [ ]:
struct Time {
    int Hours;
    int Mins;
    int Secs;
};

In [ ]:
std::ostream& operator<<(std::ostream os, const struct time_clk time)
{
    os << setfill('0') << setw(2) << static_cast<int>(time.Hours) << ":" <<
          setfill ('0') << setw(2) << static_cast<int>(time.Mins) << ":" <<
          setfill ('0') << setw(2) << static_cast<int>(time.Secs);
    
    return os;
} 

In [ ]:
int main()
{
    struct Time time;
    
    time.Hours = 11;
    time.Mins = 31;
    time.Secs = 42;
    
    cout<< "Time is " << time << endl;
    
    return 0;
}

# Ring Buffer

Pointers :
-----------------
Start => Head
End   => Tail

- Move Head at every time UART reads
- They move in a circular order

Functions:
-----------------
- Put
- Get
- Full or Not (Check)
- Empty or Not (Check)

# Ring_Buffer.h

In [ ]:
#ifndef RING_BUFFER_H
#define RING_BUFFER_H

#include <stdint>
#include <stdbool>

In [ ]:
/* Implementation is hidden inside the c code  */
/* Reference to struct */
typedef struct ring_buff_t ring_buff;

/* Pointer to type ring_buff_t which is the buff_handle */
typedef ring_buff_t* buff_handle;

In [ ]:
buff_handle Ring_Buffer_Init (uint8_t *buffer, size_t size);

In [ ]:
void Ring_Buffer_Put (buff_handle buf , uint8_t data);

In [ ]:
void Ring_Buffer_Get (buff_handle buf , uint8_t *data);

In [ ]:
void Ring_Buffer_Capacity (buff_handle buf);

In [ ]:
void Ring_Buffer_Full (buff_handle buf);

In [ ]:
void Ring_Buffer_Empty (buff_handle buf);

In [ ]:
void Ring_Buffer_Size (buff_handle buf);

In [ ]:
void Ring_Buffer_Free (buff_handle buf);

In [ ]:
void Ring_Buffer_Reset (buff_handle buf);

# Ring_Buffer.c

In [ ]:
#include <stdlib.h>
#include <stdio.h>
#include <stdint.h>
#include <stddef.h>

#include "Ring_Buffer.h"

In [ ]:
#define       RING_BUFFER_SIZE          10

struct ring_buff_t
{
    uint8_t* buffer;
    size_t head;
    size_t tail;
    size_t max;
    bool full;
};

In [ ]:
static unit8_t Ring_Buffer_Storage[RING_BUFFER_SIZE] = {0};

In [ ]:
buff_handle Ring_Buffer_Init (uint8_t *buffer, size_t size)
{
    /* Create a space for buffer handle */
    buff_handle buf = malloc(sizeof(ring_buff_t));
    
    buf -> buffer = buffer;
    buf -> max = size;
    
    Ring_Buffer_Reset();
    
    return buf;
}

In [ ]:
void Ring_Buffer_Free (buff_handle buf)
{
    free(buf);
}

In [ ]:
void Ring_Buffer_Reset (buff_handle buf)
{
    buf -> head =0;
    buf -> tail =0;
    buf -> full = false;
}

In [ ]:
void Ring_Buffer_Size (buff_handle buf)
{
    size_t size = buf -> max;
    
    if (!Ring_Buffer_Full (buf))
    {
        if (buf -> head >= buf -> tail)       // If not overflow
        {
            size = ((buf -> head) - (buf -> tail));
        }
        else                                // If there is an overflow
        {
            size = ((buf -> max + buf -> head) - buf -> tail);
        }
    }
    
    return size;
} 

In [ ]:
void Ring_Buffer_Capacity (buff_handle buf)
{
    return buf -> max;
}

In [ ]:
static void Advanced_HeadTail_Value (size_t value, size_t max)
{
    return (value +1 )% max;
}

In [ ]:
static void Advanced_Head_Pointer (buff_handle buf)
{
    if (Ring_Buffer_Full(buf))
    {
        /* Overwrite from the tail */
        buf -> tail = Advanced_HeadTail_Value (buf->tail, buf->max);
    }
    
    buf -> head = Advanced_HeadTail_Value(buf->head, buf->max);
    buf -> full = (buf->head == buf -> tail);
}

In [ ]:
void Ring_Buffer_Put (buff_handle buf , uint8_t data)
{
    buf -> buffer[buf->head] = data;
    
    Advanced_Head_Pointer(buf);
}

In [ ]:
void Ring_Buffer_Get (buff_handle buf , uint8_t *data)
{
    int r = -1;
    
    if (!Ring_Buffer_Empty(buf))
    {
        *data = buf->buffer[buf->tail];
        buf->tail = Advanced_Head_Pointer(buf->tail,buf->max);
        buf->full = false;
        r=0;
    }
    
    return r;
}

In [ ]:
void Ring_Buffer_Empty (buff_handle buf)
{
    return (!Ring_Buffer_Full(buf) && (buf->head == buf->tail));
}

In [ ]:
void Ring_Buffer_Full (buff_handle buf)
{
    return buf->full;
}

In [ ]:
int main ()
{
    buff_handle handle_ = NULL;
    int i;
    unit8_t data;
    
    handle_ = Ring_Buffer_Init(Ring_Buffer_Storage, RING_BUFFER_SIZE);
    
    int capacity = Ring_Buffer_Capacity(handle_);
    
    for (i=0; i< capacity; i++)
    {
        Ring_Buffer_Put(handle_, i);
    }
    
    printf("Size %ld", Ring_Buffer_Size(handle_));
    printf("\n Capacity %d", capacity);
    
    for (i=0; i< capacity; i++)
    {
        Ring_Buffer_Get(handle_, &data);
        printf("%d /n", data);
    }
    
    printf("Size %ld", Ring_Buffer_Size(handle_));
    printf("\n Capacity %d", capacity);
    
    return 0;
    
}

# With C++

In [ ]:
#include <iostream>
#include <string>

In [ ]:
typedef unsigned int uint;

struct Exception {
    std::string msg;
    
    Exception(const std::string &msg) : msg(msg) {}
};

In [ ]:
template<uint N, class T>
class RingBuffer {
    T buffer[N];
    uint head, tail;
    uint size, max;

public:
    RingBuffer() : head(0), tail(0), size(0), max(N) {}
    
    void push(T elem) {
        if (!isFull()) {
            if (tail < (max - 1)) {
                buffer[tail++] = elem;
            } else {
                buffer[tail] = elem;
                tail = 0;
            }
            
            size++;
        } else {
            throw Exception("Buffer is full!");
        }
    }
    
    T top() {
        return buffer[head];
    }
    
    void pop() {        
        if (!isEmpty()) {
            if (++head == max) {
                head = 0;
            }
            
            size--;
        } else {
            throw Exception("Buffer is empty!");
        }
    }
    
    bool isFull() {
        return size != 0 && head == tail;
    }
    
    bool isEmpty() {
        return size == 0;
    }
    
    template<uint M, class U>
    friend std::ostream & operator << (std::ostream &out, const RingBuffer<M, U> &buff);
};


In [ ]:
template<uint N, class T>
std::ostream & operator << (std::ostream &out, const RingBuffer<N, T> &buff) {
    uint i, j;
    
    for (i = 0, j = buff.head; i < buff.size; i++) {
        std::cout << buff.buffer[j++] << ", ";
        
        if (j == buff.max) {
            j = 0;
        }
    }
    
    std::cout << "\b\b \n";
    
    return out;
}

In [ ]:
int main() {
    RingBuffer<5, uint> buff;
    
    try {
        buff.push(1);
        buff.push(2);
        buff.push(3);
        buff.push(4);
        buff.push(5);
        
        std::cout << buff;
        
        buff.pop();
        buff.pop();
        
        std::cout << buff;
        
        buff.push(6);
        
        std::cout << buff;
        
        std::cout << "Error-free!";
    } catch (Exception &e) {
        std::cout << e.msg;
    }
    
    return 0;
}